In [1]:
import sqlite3
import pandas as pd
import os

pd.set_option('display.max_column', None)

In [2]:
# Leave that code unchanged, it is required for the server check!
db = sqlite3.connect(os.environ.get("DB_PATH") or 'database.sqlite')

In [3]:
# You may load the data from SQL table directly to the Pandas dataframe as
player_data = pd.read_sql("SELECT * FROM Player;", db)

In [4]:
player_data.head()

,id,player_api_id,player_name,player_fifa_api_id,birthday,height,weight
0,1,505942,Aaron Appindangoye,218353,1992-02-29 00:00:00,182.88,187
1,2,155782,Aaron Cresswell,189615,1989-12-15 00:00:00,170.18,146
2,3,162549,Aaron Doran,186170,1991-05-13 00:00:00,170.18,163
3,4,30572,Aaron Galindo,140161,1982-05-08 00:00:00,182.88,198
4,5,23780,Aaron Hughes,17725,1979-11-08 00:00:00,182.88,154


In [5]:
# Task 1 (0.25 point). 
# Calculate the number of players with a height between 180 and 190 inclusive

players_180_190 = player_data[(player_data['height'] >= 180) & (player_data['height'] <= 190)].shape[0]

In [6]:
assert(isinstance(players_180_190, int))
# DONE

In [7]:
# Task 2 (0.25 point). Calculate the number of players born in 1980.
# Hint: you may want to cast your 'birthday' column to DateTime type by pandas.to_datetime

player_data['birthday'] = pd.to_datetime(player_data['birthday'])
players_1980 = player_data[(player_data['birthday'] >= '1980-01-01') 
                           & (player_data['birthday'] < '1981-01-01')].shape[0]

In [8]:
assert(isinstance(players_1980, int))
# DONE

In [9]:
# Task 3 (0.25 point). Make a list of the top 10 players with the highest weight sorted in descending order. 
# If there are several players with the same weight put them in the lexicographic order by name.

df = player_data.sort_values(by=['weight', 'player_name'], ascending=[False, True])
highest_players = df.head(10)['player_name'].values.tolist()

In [10]:
assert(len(highest_players) == 10)
assert(isinstance(highest_players, list))
for i in range(10):
    assert(isinstance(highest_players[i], str))
# DONE

In [11]:
# Task 4 (0.5 point). Make a list of tuples containing years along with the number of players born in that year from 1980 up to 1990.
# Structure example: [(1980, 123), (1981, 140) ..., (1990, 83)] -> There were born 123 players in 1980, there were born 140 players in 1981 and etc.

player_data['birthday'] = pd.to_datetime(player_data['birthday'])
df = player_data
df1 = df.groupby([df['birthday'].dt.year]).agg({'count'})

new_df = df1['player_name'].copy()
df2 = new_df.loc['1980':'1990']
years_born_players = list(df2.itertuples(index=True, name=None))

In [12]:
assert(len(years_born_players) == 11)
assert(isinstance(years_born_players, list))
for i in range(10):
    assert(isinstance(years_born_players[i], tuple))
    assert(isinstance(years_born_players[i][0], int))
    assert(isinstance(years_born_players[i][1], int))
# DONE

In [13]:
# Task 5 (0.5 point). Calculate the mean and the standard deviation of the players' height with the name Adriano.
# Note: Name is represented by the first part of player_name.
df = player_data.loc[(player_data['player_name'].str.startswith('Adriano')), 
                      ['player_name','height']]

adriano_mean = df['height'].mean()
adriano_std = df['height'].std()

In [14]:
assert(isinstance(adriano_mean, float))
assert(isinstance(adriano_std, float))
# DONE

In [15]:
# Task 6 (0.75 point). How many players were born on each day of the week? Find the day of the week with the minimum number of players born.

player_data['birthday'] = pd.to_datetime(player_data['birthday'])
df = player_data
df['weekday'] = df['birthday'].dt.day_name()
day_counted = df.groupby('weekday').count()
day_sorted = day_counted.sort_values(by='id')
day_sorted['weekday'] = day_sorted.index
dow_with_min_players_born = day_sorted.head()['weekday'].values.tolist()[0]

In [16]:
assert(isinstance(dow_with_min_players_born, str))
# DONE

In [17]:
# Task 7 (0.75 point). Find a league with the most matches in total. 
# If there are several leagues with the same amount of matches, take the first in the lexical order.
match_data = pd.read_sql("SELECT id, league_id FROM Match;", db)
league_data = pd.read_sql("SELECT * FROM League;", db)

match_count_df = match_data.groupby('league_id').count()
match_count_df.reset_index(inplace=True)
match_count_df.rename(columns={'id': 'count'}, inplace=True)

df = league_data.merge(match_count_df, left_on='id', right_on='league_id', how='inner')
df = df.loc[df['count'] == 3040]
df = df.sort_values(by='name').head()
league_list = df.head()['name'].values.tolist()
league_most_matches = league_list[0]

In [18]:
assert(isinstance(league_most_matches, str))
# DONE

In [19]:
# Task 8 (1.25 point). Find a player who participated in the largest number of matches during the whole match history. 
# Assign a player_name to the given variable

match_df = pd.read_sql("SELECT id, home_player_1, home_player_2, home_player_3, home_player_4, home_player_5, home_player_6, home_player_7, home_player_8, home_player_9, home_player_10, home_player_11, away_player_1, away_player_2, away_player_3, away_player_4, away_player_5, away_player_6, away_player_7, away_player_8, away_player_9, away_player_10, away_player_11 FROM Match;", db)
player_df = pd.read_sql("SELECT player_api_id, player_name FROM Player;", db)

###
match_df1 = pd.read_sql("SELECT id, home_player_1 FROM Match;", db)
match_df_sorted1 = match_df1.sort_values(by=['home_player_1'], ascending=True)
match_df_sorted1['count_player1'] = match_df_sorted1['home_player_1'].map(match_df_sorted1['home_player_1'].value_counts())
df_drop_na = match_df_sorted1.dropna()
df_drop_dupl = df_drop_na.copy()
df_drop_dupl.drop_duplicates('home_player_1', inplace = True)
df_drop_column = df_drop_dupl.drop(['id'], axis=1)
match_count1 = df_drop_column.sort_values(by='home_player_1').set_index('home_player_1')

match_df1 = pd.read_sql("SELECT id, home_player_2 FROM Match;", db)
match_df_sorted1 = match_df1.sort_values(by=['home_player_2'], ascending=True)
match_df_sorted1['count_player2'] = match_df_sorted1['home_player_2'].map(match_df_sorted1['home_player_2'].value_counts())
df_drop_na = match_df_sorted1.dropna()
df_drop_dupl = df_drop_na.copy()
df_drop_dupl.drop_duplicates('home_player_2', inplace = True)
df_drop_column = df_drop_dupl.drop(['id'], axis=1)
match_count2 = df_drop_column.sort_values(by='home_player_2').set_index('home_player_2')
result2 = match_count1.merge(match_count2, how='left', left_index=True, right_index=True)

match_df1 = pd.read_sql("SELECT id, home_player_3 FROM Match;", db)
match_df_sorted1 = match_df1.sort_values(by=['home_player_3'], ascending=True)
match_df_sorted1['count_player3'] = match_df_sorted1['home_player_3'].map(match_df_sorted1['home_player_3'].value_counts())
df_drop_na = match_df_sorted1.dropna()
df_drop_dupl = df_drop_na.copy()
df_drop_dupl.drop_duplicates('home_player_3', inplace = True)
df_drop_column = df_drop_dupl.drop(['id'], axis=1)
match_count3 = df_drop_column.sort_values(by='home_player_3').set_index('home_player_3')
result3 = result2.merge(match_count3, how='left', left_index=True, right_index=True)

match_df1 = pd.read_sql("SELECT id, home_player_4 FROM Match;", db)
match_df_sorted1 = match_df1.sort_values(by=['home_player_4'], ascending=True)
match_df_sorted1['count_player4'] = match_df_sorted1['home_player_4'].map(match_df_sorted1['home_player_4'].value_counts())
df_drop_na = match_df_sorted1.dropna()
df_drop_dupl = df_drop_na.copy()
df_drop_dupl.drop_duplicates('home_player_4', inplace = True)
df_drop_column = df_drop_dupl.drop(['id'], axis=1)
match_count4 = df_drop_column.sort_values(by='home_player_4').set_index('home_player_4')
result4 = result3.merge(match_count4, how='left', left_index=True, right_index=True)

match_df1 = pd.read_sql("SELECT id, home_player_5 FROM Match;", db)
match_df_sorted1 = match_df1.sort_values(by=['home_player_5'], ascending=True)
match_df_sorted1['count_player5'] = match_df_sorted1['home_player_5'].map(match_df_sorted1['home_player_5'].value_counts())
df_drop_na = match_df_sorted1.dropna()
df_drop_dupl = df_drop_na.copy()
df_drop_dupl.drop_duplicates('home_player_5', inplace = True)
df_drop_column = df_drop_dupl.drop(['id'], axis=1)
match_count5 = df_drop_column.sort_values(by='home_player_5').set_index('home_player_5')
result5 = result4.merge(match_count5, how='left', left_index=True, right_index=True)

match_df1 = pd.read_sql("SELECT id, home_player_6 FROM Match;", db)
match_df_sorted1 = match_df1.sort_values(by=['home_player_6'], ascending=True)
match_df_sorted1['count_player6'] = match_df_sorted1['home_player_6'].map(match_df_sorted1['home_player_6'].value_counts())
df_drop_na = match_df_sorted1.dropna()
df_drop_dupl = df_drop_na.copy()
df_drop_dupl.drop_duplicates('home_player_6', inplace = True)
df_drop_column = df_drop_dupl.drop(['id'], axis=1)
match_count6 = df_drop_column.sort_values(by='home_player_6').set_index('home_player_6')
result6 = result5.merge(match_count6, how='left', left_index=True, right_index=True)

match_df1 = pd.read_sql("SELECT id, home_player_7 FROM Match;", db)
match_df_sorted1 = match_df1.sort_values(by=['home_player_7'], ascending=True)
match_df_sorted1['count_player7'] = match_df_sorted1['home_player_7'].map(match_df_sorted1['home_player_7'].value_counts())
df_drop_na = match_df_sorted1.dropna()
df_drop_dupl = df_drop_na.copy()
df_drop_dupl.drop_duplicates('home_player_7', inplace = True)
df_drop_column = df_drop_dupl.drop(['id'], axis=1)
match_count7 = df_drop_column.sort_values(by='home_player_7').set_index('home_player_7')
result7 = result6.merge(match_count7, how='left', left_index=True, right_index=True)

match_df1 = pd.read_sql("SELECT id, home_player_8 FROM Match;", db)
match_df_sorted1 = match_df1.sort_values(by=['home_player_8'], ascending=True)
match_df_sorted1['count_player8'] = match_df_sorted1['home_player_8'].map(match_df_sorted1['home_player_8'].value_counts())
df_drop_na = match_df_sorted1.dropna()
df_drop_dupl = df_drop_na.copy()
df_drop_dupl.drop_duplicates('home_player_8', inplace = True)
df_drop_column = df_drop_dupl.drop(['id'], axis=1)
match_count8 = df_drop_column.sort_values(by='home_player_8').set_index('home_player_8')
result8 = result7.merge(match_count8, how='left', left_index=True, right_index=True)

match_df1 = pd.read_sql("SELECT id, home_player_9 FROM Match;", db)
match_df_sorted1 = match_df1.sort_values(by=['home_player_9'], ascending=True)
match_df_sorted1['count_player9'] = match_df_sorted1['home_player_9'].map(match_df_sorted1['home_player_9'].value_counts())
df_drop_na = match_df_sorted1.dropna()
df_drop_dupl = df_drop_na.copy()
df_drop_dupl.drop_duplicates('home_player_9', inplace = True)
df_drop_column = df_drop_dupl.drop(['id'], axis=1)
match_count9 = df_drop_column.sort_values(by='home_player_9').set_index('home_player_9')
result9 = result8.merge(match_count9, how='left', left_index=True, right_index=True)

match_df1 = pd.read_sql("SELECT id, home_player_10 FROM Match;", db)
match_df_sorted1 = match_df1.sort_values(by=['home_player_10'], ascending=True)
match_df_sorted1['count_player10'] = match_df_sorted1['home_player_10'].map(match_df_sorted1['home_player_10'].value_counts())
df_drop_na = match_df_sorted1.dropna()
df_drop_dupl = df_drop_na.copy()
df_drop_dupl.drop_duplicates('home_player_10', inplace = True)
df_drop_column = df_drop_dupl.drop(['id'], axis=1)
match_count10 = df_drop_column.sort_values(by='home_player_10').set_index('home_player_10')
result10 = result9.merge(match_count10, how='left', left_index=True, right_index=True)

match_df1 = pd.read_sql("SELECT id, home_player_11 FROM Match;", db)
match_df_sorted1 = match_df1.sort_values(by=['home_player_11'], ascending=True)
match_df_sorted1['count_player11'] = match_df_sorted1['home_player_11'].map(match_df_sorted1['home_player_11'].value_counts())
df_drop_na = match_df_sorted1.dropna()
df_drop_dupl = df_drop_na.copy()
df_drop_dupl.drop_duplicates('home_player_11', inplace = True)
df_drop_column = df_drop_dupl.drop(['id'], axis=1)
match_count11 = df_drop_column.sort_values(by='home_player_11').set_index('home_player_11')
result11 = result10.merge(match_count11, how='left', left_index=True, right_index=True)

result11['total'] = result11.sum(axis=1)
sum_matches_home = result11.drop(['count_player1', 'count_player2', 'count_player3', 'count_player4',
                                  'count_player5', 'count_player6', 'count_player7', 'count_player8',
                                  'count_player9', 'count_player10', 'count_player11'], axis=1)

###
match_df1 = pd.read_sql("SELECT id, away_player_1 FROM Match;", db)
match_df_sorted1 = match_df1.sort_values(by=['away_player_1'], ascending=True)
match_df_sorted1['count_player1'] = match_df_sorted1['away_player_1'].map(match_df_sorted1['away_player_1'].value_counts())
df_drop_na = match_df_sorted1.dropna()
df_drop_dupl = df_drop_na.copy()
df_drop_dupl.drop_duplicates('away_player_1', inplace = True)
df_drop_column = df_drop_dupl.drop(['id'], axis=1)
match_count1 = df_drop_column.sort_values(by='away_player_1').set_index('away_player_1')

match_df1 = pd.read_sql("SELECT id, away_player_2 FROM Match;", db)
match_df_sorted1 = match_df1.sort_values(by=['away_player_2'], ascending=True)
match_df_sorted1['count_player2'] = match_df_sorted1['away_player_2'].map(match_df_sorted1['away_player_2'].value_counts())
df_drop_na = match_df_sorted1.dropna()
df_drop_dupl = df_drop_na.copy()
df_drop_dupl.drop_duplicates('away_player_2', inplace = True)
df_drop_column = df_drop_dupl.drop(['id'], axis=1)
match_count2 = df_drop_column.sort_values(by='away_player_2').set_index('away_player_2')
result2 = match_count1.merge(match_count2, how='left', left_index=True, right_index=True)

match_df1 = pd.read_sql("SELECT id, away_player_3 FROM Match;", db)
match_df_sorted1 = match_df1.sort_values(by=['away_player_3'], ascending=True)
match_df_sorted1['count_player3'] = match_df_sorted1['away_player_3'].map(match_df_sorted1['away_player_3'].value_counts())
df_drop_na = match_df_sorted1.dropna()
df_drop_dupl = df_drop_na.copy()
df_drop_dupl.drop_duplicates('away_player_3', inplace = True)
df_drop_column = df_drop_dupl.drop(['id'], axis=1)
match_count3 = df_drop_column.sort_values(by='away_player_3').set_index('away_player_3')
result3 = result2.merge(match_count3, how='left', left_index=True, right_index=True)

match_df1 = pd.read_sql("SELECT id, away_player_4 FROM Match;", db)
match_df_sorted1 = match_df1.sort_values(by=['away_player_4'], ascending=True)
match_df_sorted1['count_player4'] = match_df_sorted1['away_player_4'].map(match_df_sorted1['away_player_4'].value_counts())
df_drop_na = match_df_sorted1.dropna()
df_drop_dupl = df_drop_na.copy()
df_drop_dupl.drop_duplicates('away_player_4', inplace = True)
df_drop_column = df_drop_dupl.drop(['id'], axis=1)
match_count4 = df_drop_column.sort_values(by='away_player_4').set_index('away_player_4')
result4 = result3.merge(match_count4, how='left', left_index=True, right_index=True)

match_df1 = pd.read_sql("SELECT id, away_player_5 FROM Match;", db)
match_df_sorted1 = match_df1.sort_values(by=['away_player_5'], ascending=True)
match_df_sorted1['count_player5'] = match_df_sorted1['away_player_5'].map(match_df_sorted1['away_player_5'].value_counts())
df_drop_na = match_df_sorted1.dropna()
df_drop_dupl = df_drop_na.copy()
df_drop_dupl.drop_duplicates('away_player_5', inplace = True)
df_drop_column = df_drop_dupl.drop(['id'], axis=1)
match_count5 = df_drop_column.sort_values(by='away_player_5').set_index('away_player_5')
result5 = result4.merge(match_count5, how='left', left_index=True, right_index=True)

match_df1 = pd.read_sql("SELECT id, away_player_6 FROM Match;", db)
match_df_sorted1 = match_df1.sort_values(by=['away_player_6'], ascending=True)
match_df_sorted1['count_player6'] = match_df_sorted1['away_player_6'].map(match_df_sorted1['away_player_6'].value_counts())
df_drop_na = match_df_sorted1.dropna()
df_drop_dupl = df_drop_na.copy()
df_drop_dupl.drop_duplicates('away_player_6', inplace = True)
df_drop_column = df_drop_dupl.drop(['id'], axis=1)
match_count6 = df_drop_column.sort_values(by='away_player_6').set_index('away_player_6')
result6 = result5.merge(match_count6, how='left', left_index=True, right_index=True)

match_df1 = pd.read_sql("SELECT id, away_player_7 FROM Match;", db)
match_df_sorted1 = match_df1.sort_values(by=['away_player_7'], ascending=True)
match_df_sorted1['count_player7'] = match_df_sorted1['away_player_7'].map(match_df_sorted1['away_player_7'].value_counts())
df_drop_na = match_df_sorted1.dropna()
df_drop_dupl = df_drop_na.copy()
df_drop_dupl.drop_duplicates('away_player_7', inplace = True)
df_drop_column = df_drop_dupl.drop(['id'], axis=1)
match_count7 = df_drop_column.sort_values(by='away_player_7').set_index('away_player_7')
result7 = result6.merge(match_count7, how='left', left_index=True, right_index=True)

match_df1 = pd.read_sql("SELECT id, away_player_8 FROM Match;", db)
match_df_sorted1 = match_df1.sort_values(by=['away_player_8'], ascending=True)
match_df_sorted1['count_player8'] = match_df_sorted1['away_player_8'].map(match_df_sorted1['away_player_8'].value_counts())
df_drop_na = match_df_sorted1.dropna()
df_drop_dupl = df_drop_na.copy()
df_drop_dupl.drop_duplicates('away_player_8', inplace = True)
df_drop_column = df_drop_dupl.drop(['id'], axis=1)
match_count8 = df_drop_column.sort_values(by='away_player_8').set_index('away_player_8')
result8 = result7.merge(match_count8, how='left', left_index=True, right_index=True)

match_df1 = pd.read_sql("SELECT id, away_player_9 FROM Match;", db)
match_df_sorted1 = match_df1.sort_values(by=['away_player_9'], ascending=True)
match_df_sorted1['count_player9'] = match_df_sorted1['away_player_9'].map(match_df_sorted1['away_player_9'].value_counts())
df_drop_na = match_df_sorted1.dropna()
df_drop_dupl = df_drop_na.copy()
df_drop_dupl.drop_duplicates('away_player_9', inplace = True)
df_drop_column = df_drop_dupl.drop(['id'], axis=1)
match_count9 = df_drop_column.sort_values(by='away_player_9').set_index('away_player_9')
result9 = result8.merge(match_count9, how='left', left_index=True, right_index=True)

match_df1 = pd.read_sql("SELECT id, away_player_10 FROM Match;", db)
match_df_sorted1 = match_df1.sort_values(by=['away_player_10'], ascending=True)
match_df_sorted1['count_player10'] = match_df_sorted1['away_player_10'].map(match_df_sorted1['away_player_10'].value_counts())
df_drop_na = match_df_sorted1.dropna()
df_drop_dupl = df_drop_na.copy()
df_drop_dupl.drop_duplicates('away_player_10', inplace = True)
df_drop_column = df_drop_dupl.drop(['id'], axis=1)
match_count10 = df_drop_column.sort_values(by='away_player_10').set_index('away_player_10')
result10 = result9.merge(match_count10, how='left', left_index=True, right_index=True)

match_df1 = pd.read_sql("SELECT id, away_player_11 FROM Match;", db)
match_df_sorted1 = match_df1.sort_values(by=['away_player_11'], ascending=True)
match_df_sorted1['count_player11'] = match_df_sorted1['away_player_11'].map(match_df_sorted1['away_player_11'].value_counts())
df_drop_na = match_df_sorted1.dropna()
df_drop_dupl = df_drop_na.copy()
df_drop_dupl.drop_duplicates('away_player_11', inplace = True)
df_drop_column = df_drop_dupl.drop(['id'], axis=1)
match_count11 = df_drop_column.sort_values(by='away_player_11').set_index('away_player_11')
result11 = result10.merge(match_count11, how='left', left_index=True, right_index=True)

result11['total'] = result11.sum(axis=1)
sum_matches_away = result11.drop(['count_player1', 'count_player2', 'count_player3', 'count_player4',
                                  'count_player5', 'count_player6', 'count_player7', 'count_player8',
                                  'count_player9', 'count_player10', 'count_player11'], axis=1)

###
sum_matches = sum_matches_away.merge(sum_matches_home, how='left', left_index=True, right_index=True)
sum_matches['sum'] = sum_matches.sum(axis=1)
sum_matches.sort_values(by='sum', inplace=True, ascending=False)

sum_matches['player_id'] = sum_matches.index
max_matches_player_id = sum_matches.head()['player_id'].values.tolist()[0]

player_df_data = player_df[player_df['player_api_id'] == max_matches_player_id].head()
max_matches_player = player_df_data.head()['player_name'].values.tolist()[0]

In [20]:
assert(isinstance(max_matches_player, str))
# DONE

In [21]:
# Task 9 (1.5 point). List top-5 tuples of most correlated player's characteristics in the descending order of the absolute Pearson's coefficient value.

# Note 1: Players characteristics are all the columns in Player_Attributes table except [id, player_fifa_api_id, player_api_id, date, preferred_foot, attacking_work_rate, defensive_work_rate]).
# Note 2: Exclude duplicated pairs from the list. E.g. ('gk_handling', 'gk_reflexes') and ('gk_reflexes', 'gk_handling') are duplicates, leave just one of them in the resulting list.

# Hint: You may use dataframe.corr() for calculating pairwise Pearson correlation.

player_charact = pd.read_sql("SELECT * FROM Player_Attributes;", db)
player_charact = player_charact.drop(['id', 'player_fifa_api_id', 'player_api_id', 'date', 'preferred_foot', 'attacking_work_rate', 'defensive_work_rate'], axis=1)

correlation_matrix = player_charact.corr()

def get_redundant_pairs(df):
    pairs_to_drop = set()
    cols = df.columns
    for i in range(0, df.shape[1]):
        for j in range(0, i+1):
            pairs_to_drop.add((cols[i], cols[j]))
    return pairs_to_drop

def get_top_abs_correlations(df, n):
    au_corr = df.corr().abs().unstack()
    labels_to_drop = get_redundant_pairs(df)
    au_corr = au_corr.drop(labels=labels_to_drop).sort_values(ascending=False)
    return au_corr[0:n]

top_5 = get_top_abs_correlations(player_charact, 5)
top_correlated_features = top_5.index.tolist()

In [22]:
assert(len(top_correlated_features) == 5)
assert(isinstance(top_correlated_features, list))
for i in range(5):
    assert(isinstance(top_correlated_features[i], tuple))
    assert(isinstance(top_correlated_features[i][0], str))
    assert(isinstance(top_correlated_features[i][1], str))
# DONE

In [23]:
# Task 10 (2 points). Find top-5 most similar players to Neymar whose names are given. The similarity is measured as Euclidean distance between vectors of players' characteristics (described in the task above). Put their names in a vector in ascending order by Euclidean distance and sorted by player_name if the distance is the same
# Note 1: There are many records for some players in the Player_Attributes table. You need to take the freshest data (characteristics with the most recent date).
# Note 2: Use pure values of the characteristics even if you are aware of such preprocessing technics as normalization.
# Note 3: Please avoid using any built-in methods for calculating the Euclidean distance between vectors, think about implementing your own.

import numpy as np
player = pd.read_sql("SELECT player_api_id, player_name FROM Player;", db)
player_charact = pd.read_sql("SELECT * FROM Player_Attributes;", db)
player_charact = player_charact.drop(['id', 'player_fifa_api_id', 'preferred_foot', 'attacking_work_rate', 'defensive_work_rate'], axis=1)
cols = ['overall_rating','potential', 'crossing', 'finishing', 'heading_accuracy',
                         'short_passing', 'volleys', 'dribbling', 'curve', 'free_kick_accuracy', 'long_passing',
                         'ball_control', 'acceleration', 'sprint_speed', 'agility', 'reactions', 'balance',
                         'shot_power', 'jumping', 'stamina', 'strength', 'long_shots', 'aggression', 'interceptions',
                         'positioning', 'vision', 'penalties', 'marking', 'standing_tackle', 'sliding_tackle',
                         'gk_diving', 'gk_handling', 'gk_kicking', 'gk_positioning', 'gk_reflexes']

player_charact = player_charact.sort_values(by=['player_api_id', 'date'], ascending=[False, False])
player_charact_result = player_charact.groupby('player_api_id').first().reset_index()

neymar_charact = player_charact_result[player_charact_result['player_api_id'] == 19533].head()

def euclidean_dist(player_charact_result, neymar_charact, cols):
    return np.linalg.norm(player_charact_result[cols].values - neymar_charact[cols].values,
                          axis=1)

euclid_array = euclidean_dist(player_charact_result, neymar_charact, cols)
player_charact_result['euclidean_dist'] = pd.Series(euclid_array)
euclid_rate = player_charact_result.drop(cols, axis=1)
euclid_rate.sort_values(by='euclidean_dist', inplace=True)
euclid_rate_top5 = euclid_rate.iloc[1:6]

player_names = pd.merge(euclid_rate_top5, player, on='player_api_id')
neymar_similarities = player_names.head()['player_name'].values.tolist()

In [24]:
assert(len(neymar_similarities) == 5)
assert(isinstance(neymar_similarities, list))
for i in range(5):
    assert(isinstance(neymar_similarities[i], str))
# DONE

In [25]:
# Task 11 (1 point). Calculate the number of home matches played by the Borussia Dortmund team 
# in Germany 1. Bundesliga in season 2008/2009
match_df = pd.read_sql("SELECT league_id, home_team_api_id, season FROM Match;", db)
league_df = pd.read_sql("SELECT id, name FROM League;", db)
team_df = pd.read_sql("SELECT team_api_id, team_long_name FROM Team;", db)

team_name = team_df[team_df['team_long_name'] == 'Borussia Dortmund']
team_id = team_name.head()['team_api_id'].values.tolist()[0]

league_name = league_df[league_df['name'] == 'Germany 1. Bundesliga']
league_id = league_name.head()['id'].values.tolist()[0]

season_id = '2008/2009'

matches = match_df[(match_df['league_id'] == league_id) & 
                   (match_df['home_team_api_id'] == team_id) &
                   (match_df['season'] == season_id)]

borussia_bundesliga_2008_2009_matches = matches.shape[0]

In [26]:
assert(isinstance(borussia_bundesliga_2008_2009_matches, int))
# DONE

In [27]:
# Task 12 (1 point). Find a team having the most matches (both home and away!) in the Germany 1. Bundesliga in 2008/2009 season. Return number of matches.
match_df = pd.read_sql("SELECT league_id, home_team_api_id, away_team_api_id, season FROM Match;", db)
league_df = pd.read_sql("SELECT id, name FROM League;", db)
team_df = pd.read_sql("SELECT team_api_id, team_long_name FROM Team;", db)

league_name = league_df[league_df['name'] == 'Germany 1. Bundesliga']
league_id = league_name.head()['id'].values.tolist()[0]

season_id = '2008/2009'

all_matches = match_df[(match_df['league_id'] == league_id) & 
                       (match_df['season'] == season_id)]

home_matches = all_matches.groupby('home_team_api_id').count().sort_values(by='league_id')
guest_matches = all_matches.groupby('away_team_api_id').count().sort_values(by='league_id')

home_matches_count = home_matches.head(10)['league_id'].values.tolist()[0]
guest_matches_count = guest_matches.head(10)['league_id'].values.tolist()[0]
team_most_matches_bundesliga_2008_2009 = home_matches_count + guest_matches_count

In [28]:
assert(isinstance(team_most_matches_bundesliga_2008_2009, int))
# DONE

In [29]:
# Task 13 (1 point). Count total number of Arsenal matches (both home and away!) in the 2015/2016 season 
# which they have won.
# Note: Winning a game means scoring more goals than an opponent.
import numpy as np

match_df = pd.read_sql("SELECT home_team_api_id, away_team_api_id, season, home_team_goal, away_team_goal FROM Match;", db)
league_df = pd.read_sql("SELECT id, name FROM League;", db)
team_df = pd.read_sql("SELECT team_api_id, team_long_name FROM Team;", db)

team_name = team_df[team_df['team_long_name'] == 'Arsenal']
team_id = team_name.head()['team_api_id'].values.tolist()[0]

season_id = '2015/2016'

home_matches = match_df[((match_df['home_team_api_id']) == team_id) &
                         (match_df['season'] == season_id)]
away_matches = match_df[((match_df['away_team_api_id']) == team_id) &
                         (match_df['season'] == season_id)]

home_matches_results = home_matches.copy()
away_matches_results = away_matches.copy()

conditions = [home_matches_results['home_team_goal'] > home_matches_results['away_team_goal'], 
              home_matches_results['home_team_goal'] < home_matches_results['away_team_goal']]
choices = ['win', 'loose']
home_matches_results['result'] = np.select(conditions, choices, default='tie')

conditions = [away_matches_results['home_team_goal'] < away_matches_results['away_team_goal'], 
              away_matches_results['home_team_goal'] > away_matches_results['away_team_goal']]
choices = ['win', 'loose']
away_matches_results['result'] = np.select(conditions, choices, default='tie')

won_home_matches = home_matches_results[home_matches_results['result'] == 'win']
won_away_matches = away_matches_results[away_matches_results['result'] == 'win']

won_home_matches_count = won_home_matches.shape[0]
won_away_matches_count = won_away_matches.shape[0]

arsenal_won_matches_2015_2016 = won_home_matches_count + won_away_matches_count

In [30]:
assert(isinstance(arsenal_won_matches_2015_2016, int))
# DONE

In [31]:
# Task 14 (2 points). Find a team with the highest win rate in the 2015/2016 season. 
# Win rate means won matches / all matches. 
# If there are several teams with the highest win rate return the first by name in lexical order

import numpy as np

match_df = pd.read_sql("SELECT league_id, home_team_api_id, away_team_api_id, season, home_team_goal, away_team_goal FROM Match;", db)
league_df = pd.read_sql("SELECT id, name FROM League;", db)
team_df = pd.read_sql("SELECT team_api_id, team_long_name FROM Team;", db)

season_id = '2015/2016'
all_matches = match_df[(match_df['season'] == season_id)]

matches_results = all_matches.copy()

conditions = [matches_results['home_team_goal'] > matches_results['away_team_goal'], 
              matches_results['home_team_goal'] < matches_results['away_team_goal']]
choices = ['win', 'loss']
matches_results['result'] = np.select(conditions, choices, default='tie')

matches_results['home_matches_count'] = matches_results.groupby(['home_team_api_id'])['home_team_goal'].transform('count')
matches_results['away_matches_count'] = matches_results.groupby(['away_team_api_id'])['away_team_goal'].transform('count')
matches_results['matches_count'] = matches_results['home_matches_count'] + matches_results['away_matches_count']
matches_count_df = matches_results.drop(columns=['home_matches_count', 'away_matches_count'])

win_matches_count = matches_count_df[matches_count_df['result'] == 'win']
# win_matches_count

win_home_count = win_matches_count.copy()
win_home_count['home_matches_win'] = win_home_count.groupby(['home_team_api_id'])['result'].transform('count')
win_home_count['away_matches_count'] = win_home_count.groupby(['away_team_api_id'])['result'].transform('count')
win_home_count['win_matches_count'] = win_home_count['home_matches_win'] + win_home_count['away_matches_count']
win_home_count = win_home_count.drop(columns=['home_matches_win', 'away_matches_count'])
win_home_count['win_ratio'] = win_home_count['win_matches_count'] / win_home_count['matches_count']

win_home_count.sort_values(by='win_ratio', inplace=True, ascending=False)

best_team_id = win_home_count.head(10)['home_team_api_id'].values.tolist()[0]

team_highest_winrate_2015_2016 = team_df[team_df['team_api_id'] == best_team_id].values.tolist()[0][1]

In [32]:
assert(isinstance(team_highest_winrate_2015_2016, str))
# DONE

In [33]:
# Task 15 (2 points). Determine the team with the maximum days' gap between matches 
# in England Premier League 2010/2011 season. Return number of days in that gap.
# Note: a gap means the number of days between two consecutive matches of the same team.

import numpy as np

match_df = pd.read_sql("SELECT league_id, date, home_team_api_id, away_team_api_id, season FROM Match;", db)
league_df = pd.read_sql("SELECT id, name FROM League;", db)
team_df = pd.read_sql("SELECT team_api_id, team_long_name FROM Team;", db)

league_name = league_df[league_df['name'] == 'England Premier League']
league_id = league_name.head()['id'].values.tolist()[0]

season_id = '2010/2011'

matches = match_df[(match_df['league_id'] == league_id) &
                   (match_df['season'] == season_id)]
# matches
home_matches = pd.DataFrame([matches.home_team_api_id, matches.date]).transpose()
home_matches = home_matches.reset_index(drop=True)
home_matches = home_matches.rename(columns={'home_team_api_id': 'team_id', 'date': 'date'})

away_matches = pd.DataFrame([matches.away_team_api_id, matches.date]).transpose()
away_matches = away_matches.reset_index(drop=True)
away_matches = away_matches.rename(columns={'away_team_api_id': 'team_id', 'date': 'date'})

all_matches = pd.concat([home_matches, away_matches])
all_matches.sort_values(by='team_id', inplace=True)
all_matches['date'] = pd.to_datetime(all_matches['date'])
all_matches = all_matches.sort_values(by=['team_id', 'date'], ascending=[True, False])
all_matches = all_matches.reset_index(drop=True)

cols = ['date']
all_matches['gap'] = all_matches.groupby('team_id')[cols].diff(-1)

all_matches = all_matches.sort_values(by=['gap'], ascending=[False])
all_matches['gap_in_days'] = all_matches['gap'].astype('timedelta64[D]')

all_matches['gap_in_days'] = all_matches['gap_in_days'].fillna(0).astype(np.int64)

highest_gap_england_2010_2011 = all_matches.head()['gap_in_days'].values.tolist()[0]
highest_gap_england_2010_2011

18

In [34]:
assert(isinstance(highest_gap_england_2010_2011, int))
# DONE

In [35]:
with open('student_answers.txt', 'w') as file:
    file.write(f"{players_180_190}\n")
    file.write(f"{players_1980}\n")
    file.write(f"{highest_players}\n")
    file.write(f"{years_born_players}\n")
    file.write(f"{round(adriano_mean, 3)} {round(adriano_std, 3)}\n")
    file.write(f"{dow_with_min_players_born}\n")
    file.write(f"{league_most_matches}\n")
    file.write(f"{max_matches_player}\n")
    file.write(f"{';'.join(['%s,%s' % tup for tup in top_correlated_features])};\n")
    file.write(f"{neymar_similarities}\n")
    file.write(f"{borussia_bundesliga_2008_2009_matches}\n")
    file.write(f"{team_most_matches_bundesliga_2008_2009}\n")
    file.write(f"{arsenal_won_matches_2015_2016}\n")
    file.write(f"{team_highest_winrate_2015_2016}\n")
    file.write(f"{highest_gap_england_2010_2011}\n")